# Inverse Problems Exercises: s08 (non-physics)
http://physics.medma.uni-heidelberg.de/cms/

## Notes
* Please **DO NOT** change the name of the .ipynb file. 
* Please put the .ipynb file directly into the .zip-archive without any intermediate folder.

## Please provide your personal information
* full name (Name): 

Maximilian Richter

## S02b: Poisson models

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
file_gaussian = 'file_gaussian.npz'
with np.load(file_gaussian) as data:
    f_true = data['f_true']
    A_psf = data['A_psf']

### Imaging model
The imaging model can be represented by
$$
g = h \otimes f_\text{true}
= Af_\text{true}
$$
$$
g' = \operatorname{Poisson}(g).
$$
* $f_\text{true}$ is the input signal
* $h$ is the point spread function (kernel)
* $\otimes$ is the convolution operator
* $A$ is the Toeplitz matrix of $h$
* $\operatorname{Poisson}(\cdot)$ is the Poisson process operator
* $g$ is the filtered signal
* $g'$ is the noisy signal

### Poisson noise

$\operatorname{Poisson}(\cdot)$ adds the Poisson noise to the signal
* Given the input signal $g$
* Given the maximum number of deteced photons $p_{max}$
* Generate the random variable $p \sim \operatorname{Pois}(\lambda)$, with $\lambda = g / max(g) \cdot p_{max}$ (using `numpy.random.Generator.poisson()`)
* Return the scaled random variable $g' = p / p_{max} \cdot max(g)$
* Implement the function `add_poisson_noise()` (using `numpy.array`)

Create the noisy signals $g'$ as follows
* Apply the Toeplitz matrix `A_psf` to `f_true`
* add the noise with 3 different maximum detected photons $10^3$, $10^2$, $10^1$, respectively
* Save the outputs in the variable `list_gn` (as `list` of `numpy.array`)

Display the result
* Plot the outputs in `list_gn` in the same order of the parameter options in the subplots of `axs`
* Plot `f_true` in the last subplot of `axs`
* Add proper titles to the subplots of `axs`

In [ ]:
def add_poisson_noise(signal, num_photons):
    """ Add Poisson distributed noise to the signal.
    The maximum of the signal represents the given number of photons.

    The signal is scaled using the number of photons and noise is added, afterwards
    the signal is scaled back to its original range.

    :param signal: Signal to be corrupted.
    :param num_photons: Number of photons/observed interval of the Poisson distribution.
    :returns: Noisy signal.
    """
    lamb = signal/np.max(signal) * num_photons
    rng = np.random.default_rng()
    p = rng.poisson(lamb, size=g.shape)
    return p/num_photons * np.max(signal)


fig, axs = plt.subplots(4, 1, figsize = (15, 15))
fig.suptitle('Poisson noise')

g = A_psf @ f_true
list_num_photons =  [1e3, 1e2, 1e1]
list_gn = [add_poisson_noise(g, num_photons) for num_photons in list_num_photons]

for i in range(3):
    axs[i].plot(list_gn[i])
    axs[i].set_title("$p_{{max}} = {}$".format(list_num_photons[i]))
axs[3].plot(f_true)
axs[3].set_title("$f_{true}$")

In [ ]:
# This cell contains hidden tests.


In [ ]:
# This cell contains hidden tests.


In [ ]:
# This cell contains hidden tests.


### Maximum Likelihood
The objective function with the log likelihood is
$$
L(f) = \sum_i (g'_i \ln⁡(Af)_i - (Af)_i)
$$
To maximize the likelihood function (ML), the Expectation Maximization (EM) method is used, which is also known in the literature as Richardson-Lucy deconvolution. When the kernel $h$ is normalized to unity, the iterative solution is given by
$$
f^{(n+1)} = f^{(n)} \circ A^T (\frac{g'}{Af^{(n)}}),
$$
where $\circ$ and $\frac{\cdot}{\cdot}$ are element-wise multiplication and division, respectively.

Implement the objective function
* Given the estimate $f$
* Given the system matrix $A$
* Given the observed signal $g'$
* Implement the function `objective_likelihood()` (using `numpy.array`)

Implement the iterative expectation maximization updates
* Given the system matrix $A$
* Given the observed signal $g'$
* Given the initial value $f^{(0)}$
* Given the number of iterations $n$
* Return the final value $f^{(n)}$ as the first output
* Return the history array of objective values $[L(f^{(0)}), ..., L(f^{(n)})]$ as the second output
* Implement the function `solve_ml_em()` (using `numpy.array`)
  
Solve the objective function
* Calculate the solution by iterative expectation maximization updates for the noisy signals in `list_gn`
* Note the kernel of `A_psf` is already normalized
* Return the outputs with $f^{(0)}=1$, $n=10$
* Save the solutions in the variable `list_f_mlem` (as `list` of `numpy.array`)
* Save the corresponding objective value history in the variable `list_L_mlem` (as `list` of `numpy.array`)

Display the result
* Plot the outputs in `list_f_mlem` in the same order of the noisy signals in the subplots of `axs`
* Plot the corresponding noisy signal in each subplot
* Plot the input signal `f_true` in each subplot
* Show the legend in each subplot
* Show the case information in the titles to the subplots
* Show the objective function value of each output in the titles to the subplots
* Plot the arrays in `list_L_mlem` as solid lines in the same order of the noisy signals in the last subplot of `axs`
* Show the legend in the last subplot
* Add a proper title to the last subplot

In [ ]:
def objective_likelihood(f, A, gn):
    """
    :param f: Current estimate of the signal.
    :param A: 2d Matrix A of the linear problem.
    :param gn: Observed signal.
    :returns: Objective function value.
    """
    Af = A @ f
    return np.sum(gn * np.log(Af) - Af)

def solve_ml_em(A, gn, f0, n):
    """ 
    :param A: 2d Matrix A of the linear problem.
    :param gn: Observed signal.
    :param f0: Starting values for initializing the parameters f.
    :param n: Number of iterations.
    :returns: Final f values and a vector with the history of objective values.
    """
    fn = f0
    L = []
    for i in range(n):
        fn = fn * A.T @ (gn / (A@fn))
        L.append(objective_likelihood(fn, A, gn))
    return fn, L
    
fig, axs = plt.subplots(4, 1, figsize = (15, 15))
fig.suptitle('MLEM solutions')

f0 = np.ones(f_true.shape)
n = 10
list_f_mlem = []
list_L_mlem = []

for i in range(3):
    fn, L = solve_ml_em(A_psf, list_gn[i], f0, n)
    list_f_mlem.append(fn)
    list_L_mlem.append(L)

    axs[i].plot(list_f_mlem[i], label="$f^{(n)}$")
    axs[i].plot(list_gn[i], label="$g$")
    axs[i].plot(f_true, label="$f_{true}$")
    axs[i].set_title("$p_{{max}} = {}, L = {}$".format(list_num_photons[i], np.round(list_L_mlem[i][-1])))
    axs[i].legend()
    axs[3].plot(list_L_mlem[i], label="$p_{{max}} = {}$".format(list_num_photons[i]))
axs[3].legend()

In [ ]:
# This cell contains hidden tests.


In [ ]:
# This cell contains hidden tests.


In [ ]:
# This cell contains hidden tests.


### Regularization with One-Step-Late
Providing the difference matrix
$$D' = \begin{bmatrix} 
1 & 0 & 0 & 0 & ... & 0 & -1 \\
-1 & 1 & 0 & 0 & ... & 0 & 0 \\
0 & -1 & 1 & 0 & ... & 0 & 0 \\
  &   &   & ... &   &   & \\
0 & 0 & 0 & 0 & ... & -1 & 1 \end{bmatrix},$$
the Tikhonov regularization term is
$$
R(f) = \|D'f\|_2^2.
$$
The gradient of the Tikhonov regularization is
$$
\nabla R(f) = D'^T D'f.
$$
With the One-Step-Late (OSL) method, the iterative solution with the regularization term is given by
$$
f^{(n+1)} = \frac{f^{(n)}}{1 + \lambda \nabla R(f^{(n)})} \circ A^T (\frac{g'}{Af^{(n)}}),
$$
where $\circ$ and $\frac{\cdot}{\cdot}$ are element-wise multiplication and division, respectively.

Implement the gradient of the Tikhonov regularization
* Given the estimate $f$
* Implement the function `gradient_tikhonov_reg()` (using `numpy.array`)

Implement the iterative expectation maximization updates with OSL
* Given the system matrix $A$
* Given the observed signal $g'$
* Given the regularization parameter $\lambda$
* Given the initial value $f^{(0)}$
* Given the number of iterations $n$
* Return the final value $f^{(n)}$ as the first output
* Return the history array of objective values $[L(f^{(0)}), ..., L(f^{(n)})]$ as the second output
* Implement the function `solve_ml_em_osl()` (using `numpy.array`)
  
Solve the objective function
* Calculate the solution by iterative expectation maximization updates with OSL for the noisy signals in `list_gn`
* Return the outputs with $\lambda = 0.001$, $f^{(0)}=1$, $n=10$
* Save the solutions in the variable `list_f_osl` (as `list` of `numpy.array`)
* Save the corresponding objective value history in the variable `list_L_osl` (as `list` of `numpy.array`)

Display the result
* Plot the outputs in `list_f_osl` in the same order of the noisy signals in the subplots of `axs`
* Plot the corresponding noisy signal in each subplot
* Plot the input signal `f_true` in each subplot
* Show the legend in each subplot
* Show the case information in the titles to the subplots
* Show the objective function value of each output in the titles to the subplots
* Plot the arrays in `list_L_osl` as solid lines in the same order of the noisy signals in the last subplot of `axs`
* Show the legend in the last subplot
* Add a proper title to the last subplot

In [ ]:
def get_diff_matrix(n):
    """ Compute a matrix to calculate the difference along a vector of the size n
    between two neighboring elements.

    :param n: Size of the target vector.
    :returns: Matrix with shape (n, n), which calculates the difference.
    """
    diff_matrix = np.eye(n)
    diff_matrix[1:,:-1] += np.eye(n-1)*-1
    diff_matrix[0,n-1] = -1
    return diff_matrix

def gradient_tikhonov_reg(f):
    """ 
    :param f: Current estimate of the signal.
    :returns: Gradient value of the Tikhonov regularization term.
    """
    D = get_diff_matrix(f.shape[0])
    return D.T @ D @ f
    
def solve_ml_em_osl(A, gn, lb, f0, n):
    """ 
    :param A: Matrix of the linear problem g = A⋅f
    :param gn: Observed signal.
    :param lb: Regularization strength.
    :param f0: Starting values for initializing the parameters f.
    :param n: Number of iterations.
    :returns: Final f values and a vector with the history of objective values.
    """
    fn = f0
    L = []
    for i in range(n):
        fn = fn/(1 + lb*gradient_tikhonov_reg(fn)) * A.T @ (gn / (A@fn))
        L.append(objective_likelihood(fn, A, gn))
    return fn, L
    
fig, axs = plt.subplots(4, 1, figsize = (15, 15))
fig.suptitle('OSL solutions')

f0 = np.ones(f_true.shape)
n = 10
lb = 0.001
list_f_osl = []
list_L_osl = []

for i in range(3):
    fn, L = solve_ml_em_osl(A_psf, list_gn[i], lb, f0, n)
    list_f_osl.append(fn)
    list_L_osl.append(L)

    axs[i].plot(list_f_osl[i], label="$f^{(n)}$")
    axs[i].plot(list_gn[i], label="$g$")
    axs[i].plot(f_true, label="$f_{true}$")
    axs[i].set_title("$p_{{max}} = {}, L = {}$".format(list_num_photons[i], np.round(list_L_osl[i][-1])))
    axs[i].legend()
    axs[3].plot(list_L_osl[i], label="$p_{{max}} = {}$".format(list_num_photons[i]))
axs[3].legend()

In [ ]:
# This cell contains hidden tests.


In [ ]:
# This cell contains hidden tests.


In [ ]:
# This cell contains hidden tests.
